# Read business columns (CSV → Delta) — minimal view
This notebook reads a **small, business‑critical subset of columns** from your data.

You can either:
- **Read the specific Parquet part file** you mentioned (fast projection)
- *(Optional)* Read the **entire Delta table** and then select the same columns

> **Note:** Keep only a handful of columns so they fit nicely in the VS Code terminal output.


In [ ]:
# ✅ Imports & version check
import sys, os
import pandas as pd

try:
    import pyarrow, pyarrow.parquet as pq
except Exception as e:
    raise RuntimeError("PyArrow is required. Install with: python -m pip install pyarrow==21.0.0")

try:
    from deltalake import DeltaTable
except Exception:
    DeltaTable = None  # Optional for reading a full Delta table

print('Python:', sys.executable)
print('pandas:', pd.__version__)
print('pyarrow:', pyarrow.__version__)
print('deltalake:', end=' ')
print(DeltaTable.__module__.split('.')[0] if DeltaTable else 'not installed')


In [ ]:
# 📁 Set your paths
PARQUET_FILE = r"C:\Users\mohanreddykotha\sumitomo_datalake\sumitomo\data\bronze_delta\Slovakia\REPORT_X_SLOVAKIA__Sheet1\part-00001-3e05bdd7-3870-4ec7-8f3a-e6bf0b6c053c-c000.snappy.parquet"
TABLE_DIR    = r"C:\Users\mohanreddykotha\sumitomo_datalake\sumitomo\data\bronze_delta\Slovakia\REPORT_X_SLOVAKIA__Sheet1"

assert os.path.exists(PARQUET_FILE), f'Parquet file not found: {PARQUET_FILE}'
assert os.path.isdir(TABLE_DIR), f'Table directory not found: {TABLE_DIR}'
PARQUET_FILE, TABLE_DIR


In [ ]:
# 🧾 Business columns — small subset only (UPPERCASE names)
BUSINESS_KEEP = [
    'ORDER_NO','MODEL_2','ORDER_DATE','VIN','MODEL_YEAR',
    'BODY','SERIES','ENGINE','TRANSMISSION','COLOUR','TRIM','OPTION_LIST'
]
len(BUSINESS_KEEP), BUSINESS_KEEP


In [ ]:
# ▶️ A) Read the **single Parquet file** with projection (fast)
#    We first inspect the schema, intersect with BUSINESS_KEEP, then read only those columns.

import pyarrow.parquet as pq
schema = pq.read_schema(PARQUET_FILE)
available = {str(f.name) for f in schema}
keep_final = [c for c in BUSINESS_KEEP if c in available]
missing = [c for c in BUSINESS_KEEP if c not in available]

df = pd.read_parquet(PARQUET_FILE, columns=keep_final, engine='pyarrow')

print('Columns requested :', len(BUSINESS_KEEP))
print('Columns read      :', len(keep_final))
if missing:
    print('Missing columns :', missing)

pd.set_option('display.max_columns', None, 'display.width', 200)
print('
=== Business columns (first 20 rows) ===')
print(df.head(20).to_string(index=False))
df.head()


---
### (Optional) B) Read the **entire Delta table** and then select the same columns
Use this if you want rows across **all** Parquet parts. Requires the `deltalake` package.


In [ ]:
if DeltaTable is not None:
    dt = DeltaTable(TABLE_DIR)
    print('Delta version:', dt.version())
    df_all = dt.to_pandas()
    cols_in_data = set(df_all.columns)
    keep_final = [c for c in BUSINESS_KEEP if c in cols_in_data]
    missing = [c for c in BUSINESS_KEEP if c not in cols_in_data]
    df_tbl = df_all[keep_final] if keep_final else df_all
    print('Columns requested :', len(BUSINESS_KEEP))
    print('Columns read      :', len(keep_final))
    if missing:
        print('Missing columns :', missing)
    print('
=== Business columns from FULL TABLE (first 20 rows) ===')
    print(df_tbl.head(20).to_string(index=False))
else:
    print('deltalake not installed; skipping full-table read.')
